# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy as db
from sqlalchemy import create_engine

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words("english")

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV



#from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier

from joblib import dump, load
import pickle


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\erika.gintautas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\erika.gintautas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\erika.gintautas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# load data from database
engine = create_engine('sqlite:///web_app/data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)

X = df.iloc[:,1]
Y = df.iloc[:,4:40]


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return clean_tokens

In [9]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


I noticed that colunn related is 2, when all other columns are 0. But the same holds true for realted == 0. I will replace related = 2 into 0 in order do have a binary classification

In [10]:
df[df.related == 0].iloc[:,5:40].sum()

request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report             0
dtype: int64

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000014820DFDBF8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
def print_class_report(y_test, y_pred, colnames):
    for i,col in enumerate(colnames):
        print(col)
        print(classification_report(y_test[:,i], y_pred[:,i]))
        print('\n')
    pass

colnames = Y.columns
print_class_report(y_test.values, y_pred, colnames)

related
              precision    recall  f1-score   support

           0       0.64      0.51      0.57      1610
           1       0.85      0.90      0.88      4944

    accuracy                           0.81      6554
   macro avg       0.74      0.71      0.72      6554
weighted avg       0.80      0.81      0.80      6554



request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5443
           1       0.80      0.51      0.63      1111

    accuracy                           0.90      6554
   macro avg       0.86      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554



offer


c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554



aid_related
              precision    recall  f1-score   support

           0       0.80      0.81      0.80      3884
           1       0.72      0.70      0.71      2670

    accuracy                           0.77      6554
   macro avg       0.76      0.76      0.76      6554
weighted avg       0.77      0.77      0.77      6554



medical_help
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      6019
           1       0.65      0.18      0.28       535

    accuracy                           0.93      6554
   macro avg       0.79      0.58      0.62      6554
weighted avg       0.91      0.93      0.90  

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh



aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6473
           1       0.00      0.00      0.00        81

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554



other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6271
           1       0.33      0.00      0.01       283

    accuracy                           0.96      6554
   macro avg       0.65      0.50      0.49      6554
weighted avg       0.93      0.96      0.94      6554



weather_related
              precision    recall  f1-score   support

           0       0.91      0.94      0.92      4781
           1       0.82      0.75      0.78      1773

    accuracy                           0.89      6554
   macro avg       0.86      0.84      0.85      6554
weighted avg       

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6035
           1       0.88      0.53      0.66       519

    accuracy                           0.96      6554
   macro avg       0.92      0.76      0.82      6554
weighted avg       0.95      0.96      0.95      6554



storm
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      5949
           1       0.72      0.64      0.68       605

    accuracy                           0.94      6554
   macro avg       0.84      0.81      0.82      6554
weighted avg       0.94      0.94      0.94      6554



fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6488
           1       0.33      0.02      0.03        66

    accuracy                           0.99      6554
   macro avg       0.66      0.51      0.51      6554
weighted avg       0.98      0.99      0.99      6554



ea

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],
        #'features__transformer_weights': (
        #    {'text_pipeline': 1, 'starting_verb': 0.5},
        #    {'text_pipeline': 0.5, 'starting_verb': 1},
        #    {'text_pipeline': 0.8, 'starting_verb': 1},
        #)
    }

cv = GridSearchCV(pipeline, parameters, verbose = 3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize, ngram_range = (1,1))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [16]:
y_pred = pipeline.predict(X_test)
colnames = Y.columns
print_class_report(y_test.values, y_pred, colnames)

related
             precision    recall  f1-score   support

          0       0.60      0.51      0.55      1526
          1       0.86      0.90      0.88      5028

avg / total       0.80      0.81      0.80      6554



request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5399
          1       0.80      0.44      0.57      1155

avg / total       0.88      0.88      0.87      6554



offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6519
          1       0.00      0.00      0.00        35

avg / total       0.99      0.99      0.99      6554



aid_related
             precision    recall  f1-score   support

          0       0.74      0.86      0.80      3785
          1       0.76      0.59      0.66      2769

avg / total       0.75      0.75      0.74      6554



medical_help
             precision    recall  f1-score   support

          0       0.93      0.9

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [39]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])
parameters = {
        #'vect__ngram_range': ((1, 1)),
        #'vect__max_df': (0.5),
        'vect__max_features': (950, 1900),
        'tfidf__use_idf': (True, False),
        #'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],
        #'features__transformer_weights': (
        #    {'text_pipeline': 1, 'starting_verb': 0.5},
        #    {'text_pipeline': 0.5, 'starting_verb': 1},
        #    {'text_pipeline': 0.8, 'starting_verb': 1},
        #)
    }

cv = GridSearchCV(pipeline, parameters, verbose = 3)
cv.fit(X_train, y_train)
print(cv.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__max_features=950; total time= 4.7min
[CV 2/5] END clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__max_features=950; total time= 6.3min
[CV 3/5] END clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__max_features=950; total time= 4.7min
[CV 4/5] END clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__max_features=950; total time= 4.6min
[CV 5/5] END clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__max_features=950; total time= 5.7min
[CV 1/5] END clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__max_features=1900; total time= 7.9min
[CV 2/5] END clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__max_features=1900; total time= 6.2min
[CV 3/5] END clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__max_features=1900; total time= 9.8min
[CV 4/5] END clf__estima

In [40]:
y_pred = cv.predict(X_test)
colnames = Y.columns
print_class_report(y_test.values, y_pred, colnames)

related
              precision    recall  f1-score   support

           0       0.67      0.51      0.58      1594
           1       0.85      0.92      0.89      4960

    accuracy                           0.82      6554
   macro avg       0.76      0.71      0.73      6554
weighted avg       0.81      0.82      0.81      6554



request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5406
           1       0.80      0.55      0.65      1148

    accuracy                           0.90      6554
   macro avg       0.86      0.76      0.79      6554
weighted avg       0.89      0.90      0.89      6554



offer


c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6514
           1       0.00      0.00      0.00        40

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554



aid_related
              precision    recall  f1-score   support

           0       0.80      0.83      0.81      3795
           1       0.75      0.72      0.73      2759

    accuracy                           0.78      6554
   macro avg       0.78      0.77      0.77      6554
weighted avg       0.78      0.78      0.78      6554



medical_help
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      6028
           1       0.60      0.14      0.23       526

    accuracy                           0.92      6554
   macro avg       0.76      0.57      0.59      6554
weighted avg       0.90      0.92      0.90  

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554



aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6486
           1       0.00      0.00      0.00        68

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554



other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6278
           1       0.00      0.00      0.00       276

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96    

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

### Found best setting!

In [42]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize, max_features = 950)),
    ('tfidf', TfidfTransformer(use_idf = False)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split = 3,random_state=42 )))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
colnames = Y.columns
print_class_report(y_test.values, y_pred, colnames)


related
              precision    recall  f1-score   support

           0       0.66      0.50      0.57      1594
           1       0.85      0.92      0.88      4960

    accuracy                           0.81      6554
   macro avg       0.75      0.71      0.72      6554
weighted avg       0.80      0.81      0.81      6554



request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5406
           1       0.79      0.55      0.65      1148

    accuracy                           0.90      6554
   macro avg       0.85      0.76      0.79      6554
weighted avg       0.89      0.90      0.89      6554



offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6514
           1       0.00      0.00      0.00        40

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99     

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554



aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6486
           1       0.00      0.00      0.00        68

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554



other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6278
           1       0.00      0.00      0.00       276

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96    

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))





storm
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      5971
           1       0.74      0.68      0.71       583

    accuracy                           0.95      6554
   macro avg       0.86      0.83      0.84      6554
weighted avg       0.95      0.95      0.95      6554



fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6488
           1       0.00      0.00      0.00        66

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554



earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5925
           1       0.90      0.79      0.84       629

    accuracy                           0.97      6554
   macro avg       0.94      0.89      0.91      6554
weighted avg       0.97      0.97      0.97  

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6225
           1       0.43      0.07      0.12       329

    accuracy                           0.95      6554
   macro avg       0.69      0.53      0.55      6554
weighted avg       0.93      0.95      0.93      6554



direct_report
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      5282
           1       0.76      0.44      0.56      1272

    accuracy                           0.86      6554
   macro avg       0.82      0.70      0.74      6554
weighted avg       0.86      0.86      0.85      6554





In [17]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])
parameters = {
        #'vect__ngram_range': ((1, 1)),
        #'vect__max_df': (0.5),
        'vect__max_features': [950],
        'tfidf__use_idf': [False],
        'clf__estimator__n_estimators': [10, 100, 200],
        'clf__estimator__min_samples_split': [ 4, 0.05, 6],
        #'features__transformer_weights': (
        #    {'text_pipeline': 1, 'starting_verb': 0.5},
        #    {'text_pipeline': 0.5, 'starting_verb': 1},
        #    {'text_pipeline': 0.8, 'starting_verb': 1},
        #)
    }

cv = GridSearchCV(pipeline, parameters, verbose = 3)
cv.fit(X_train, y_train)
print(cv.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_features=950; total time=  27.7s
[CV 2/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_features=950; total time=  27.4s
[CV 3/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_features=950; total time=  28.0s
[CV 4/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_features=950; total time=  27.3s
[CV 5/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__max_features=950; total time=  26.7s
[CV 1/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=100, tfidf__use_idf=False, vect__max_features=950; total time= 3.6min
[CV 2/5] END clf__estimator__min_samples_split=4, c

In [26]:
y_pred = cv.predict(X_test)
colnames = Y.columns
print_class_report(y_test.values, y_pred, colnames)

related
              precision    recall  f1-score   support

           0       0.65      0.52      0.58      1586
           1       0.86      0.91      0.88      4968

    accuracy                           0.82      6554
   macro avg       0.75      0.72      0.73      6554
weighted avg       0.81      0.82      0.81      6554



request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5491
           1       0.79      0.52      0.63      1063

    accuracy                           0.90      6554
   macro avg       0.85      0.75      0.78      6554
weighted avg       0.89      0.90      0.89      6554



offer


c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6518
           1       0.00      0.00      0.00        36

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554



aid_related
              precision    recall  f1-score   support

           0       0.81      0.82      0.82      3877
           1       0.74      0.72      0.73      2677

    accuracy                           0.78      6554
   macro avg       0.77      0.77      0.77      6554
weighted avg       0.78      0.78      0.78      6554



medical_help
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      6067
           1       0.56      0.18      0.28       487

    accuracy                           0.93      6554
   macro avg       0.75      0.59      0.62      6554
weighted avg       0.91      0.93      0.91  

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554



water
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      6160
           1       0.82      0.53      0.65       394

    accuracy                           0.97      6554
   macro avg       0.90      0.76      0.81      6554
weighted avg       0.96      0.97      0.96      6554



food
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      5853
           1       0.84      0.75      0.79       701

    accuracy                           0.96      6554
   macro avg       0.90      0.86      0.88      6554
weighted avg       0.96      0.96      0.96      6554



shelter
              precision    recall  f1-score   su

...

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6517
           1       0.00      0.00      0.00        37

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554



hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6484
           1       1.00      0.01      0.03        70

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.98      6554



shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6519
           1       0.00      0.00      0.00        35

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554


c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           1       0.00      0.00      0.00        80

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554



other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6260
           1       0.00      0.00      0.00       294

    accuracy                           0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.95      0.93      6554



weather_related
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      4733
           1       0.82      0.73      0.77      1821

    accuracy                           0.88      6554
   macro avg       0.86      0.83      0.85      6554
weighted avg       0.88      0.88

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh




cold
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6426
           1       0.70      0.15      0.25       128

    accuracy                           0.98      6554
   macro avg       0.84      0.57      0.62      6554
weighted avg       0.98      0.98      0.98      6554



other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6214
           1       0.54      0.06      0.12       340

    accuracy                           0.95      6554
   macro avg       0.74      0.53      0.54      6554
weighted avg       0.93      0.95      0.93      6554



direct_report
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      5316
           1       0.75      0.38      0.51      1238

    accuracy                           0.86      6554
   macro avg       0.81      0.68      0.71      6554
weighted avg       0.85      0.86 

In [20]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize, max_features = 950)),
    ('tfidf', TfidfTransformer(use_idf = False)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split = 6, n_estimators = 200,random_state=42 )))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
colnames = Y.columns
print_class_report(y_test.values, y_pred, colnames)

related
              precision    recall  f1-score   support

           0       0.64      0.51      0.57      1610
           1       0.85      0.90      0.88      4944

    accuracy                           0.81      6554
   macro avg       0.74      0.71      0.72      6554
weighted avg       0.80      0.81      0.80      6554



request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5443
           1       0.80      0.51      0.63      1111

    accuracy                           0.90      6554
   macro avg       0.86      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554



offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99     

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      6019
           1       0.65      0.18      0.28       535

    accuracy                           0.93      6554
   macro avg       0.79      0.58      0.62      6554
weighted avg       0.91      0.93      0.90      6554



medical_products
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6210
           1       0.71      0.18      0.29       344

    accuracy                           0.95      6554
   macro avg       0.83      0.59      0.63      6554
weighted avg       0.94      0.95      0.94      6554



search_and_rescue
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6395
           1       0.56      0.06      0.10       159

    accuracy                           0.98      6554
   macro avg       0.77      0.53      0.55      6554
weighted avg       0.97      0.98  

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6498
           1       0.00      0.00      0.00        56

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554



shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6530
           1       0.00      0.00      0.00        24

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554



aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6473
           1       0.00      0.00      0.00        81

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      655

### 8. Try improving my model further. Here are a few ideas:
* try other machine learning algorithms --> Tried Kmeans
* add other features besides the TF-IDF --> Used 

In [ ]:
parameters = {
        'vect__ngram_range': ((1, 1)),
        'vect__max_df': ( 1.0),
        'vect__max_features': (None, 1900, 500, 950),
        'tfidf__use_idf': (True),
        #'clf__estimator__n_estimators': [10, 50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],
        #'features__transformer_weights': (
        #    {'text_pipeline': 1, 'starting_verb': 0.5},
        #    {'text_pipeline': 0.5, 'starting_verb': 1},
        #    {'text_pipeline': 0.8, 'starting_verb': 1},
        #)
    }
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(random_state=42)))
])
pipeline.get_params()
#cv = GridSearchCV(pipeline, parameters, verbose = 3)
#cv.fit(X_train, y_train)



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)
colnames = Y.columns
print_class_report(y_test.values, y_pred, colnames)

In [31]:
parameters = {
        'vect__ngram_range': ((1, 1), (1,2)),
        'vect__max_df': [1.0],
        'vect__max_features': (None, 1900, 500, 100),
        'tfidf__use_idf': (True, False),
        #'clf__estimator__n_estimators': [10, 50, 100, 200],
        'clf__estimator__leaf_size': [30, 100, 1000],
        #'features__transformer_weights': (
        #    {'text_pipeline': 1, 'starting_verb': 0.5},
        #    {'text_pipeline': 0.5, 'starting_verb': 1},
        #    {'text_pipeline': 0.8, 'starting_verb': 1},
        #)
    }
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(random_state=42)))
])

cv = GridSearchCV(pipeline, parameters, verbose = 3)

In [32]:
cv = GridSearchCV(pipeline, parameters, verbose = 3)
cv.fit(X_train, y_train)
print(cv.best_params_)
y_pred = cv.predict(X_test)
colnames = Y.columns
print_class_report(y_test.values, y_pred, colnames)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END clf__estimator__leaf_size=30, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1); total time= 1.9min
[CV 2/5] END clf__estimator__leaf_size=30, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1); total time= 1.5min
[CV 3/5] END clf__estimator__leaf_size=30, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1); total time= 1.6min
[CV 4/5] END clf__estimator__leaf_size=30, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1); total time= 1.3min
[CV 5/5] END clf__estimator__leaf_size=30, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1); total time= 1.3min
[CV 1/5] END clf__estimator__leaf_size=30, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 2); total time= 1.3min
[CV 2/5] END clf__estimato

[CV 5/5] END clf__estimator__leaf_size=30, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 1); total time= 1.3min
[CV 1/5] END clf__estimator__leaf_size=30, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 2); total time= 1.3min
[CV 2/5] END clf__estimator__leaf_size=30, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 2); total time= 1.3min
[CV 3/5] END clf__estimator__leaf_size=30, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 2); total time= 1.2min
[CV 4/5] END clf__estimator__leaf_size=30, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 2); total time= 1.3min
[CV 5/5] END clf__estimator__leaf_size=30, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 2); total time= 1.3min
[CV 1/5] END clf__estimator__leaf_size=30, tfidf__use_idf=False, vect__max_df=1.0,

[CV 4/5] END clf__estimator__leaf_size=100, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=500, vect__ngram_range=(1, 2); total time= 1.3min
[CV 5/5] END clf__estimator__leaf_size=100, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=500, vect__ngram_range=(1, 2); total time= 1.3min
[CV 1/5] END clf__estimator__leaf_size=100, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=100, vect__ngram_range=(1, 1); total time= 1.2min
[CV 2/5] END clf__estimator__leaf_size=100, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=100, vect__ngram_range=(1, 1); total time= 1.2min
[CV 3/5] END clf__estimator__leaf_size=100, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=100, vect__ngram_range=(1, 1); total time= 1.2min
[CV 4/5] END clf__estimator__leaf_size=100, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=100, vect__ngram_range=(1, 1); total time= 1.2min
[CV 5/5] END clf__estimator__leaf_size=100, tfidf__use_idf=True, vect__max_df=1.0, vect_

[CV 3/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1); total time= 1.2min
[CV 4/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1); total time= 1.3min
[CV 5/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1); total time= 1.2min
[CV 1/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 2); total time= 1.2min
[CV 2/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 2); total time= 1.3min
[CV 3/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 2); total time= 1.2min
[CV 4/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=True, vect__max_

[CV 2/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 2); total time= 1.3min
[CV 3/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 2); total time= 1.2min
[CV 4/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 2); total time= 1.2min
[CV 5/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=1900, vect__ngram_range=(1, 2); total time= 1.2min
[CV 1/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=500, vect__ngram_range=(1, 1); total time= 1.2min
[CV 2/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=500, vect__ngram_range=(1, 1); total time= 1.2min
[CV 3/5] END clf__estimator__leaf_size=1000, tfidf__use_idf=False, vect_

...

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6259
           1       0.00      0.00      0.00       295

    accuracy                           0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.95      0.93      6554



buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6204
           1       0.79      0.03      0.06       350

    accuracy                           0.95      6554
   macro avg       0.87      0.52      0.52      6554
weighted avg       0.94      0.95      0.92      6554



electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6432
           1       0.00      0.00      0.00       122

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.96      0.98      0.97     

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh




weather_related
              precision    recall  f1-score   support

           0       0.76      0.98      0.86      4797
           1       0.79      0.17      0.28      1757

    accuracy                           0.77      6554
   macro avg       0.78      0.58      0.57      6554
weighted avg       0.77      0.77      0.70      6554



floods
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6025
           1       1.00      0.01      0.02       529

    accuracy                           0.92      6554
   macro avg       0.96      0.51      0.49      6554
weighted avg       0.93      0.92      0.88      6554



storm
              precision    recall  f1-score   support

           0       0.92      0.99      0.96      5971
           1       0.67      0.11      0.18       583

    accuracy                           0.92      6554
   macro avg       0.79      0.55      0.57      6554
weighted avg       0.90      0.92     

c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\erika.gintautas\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

In [41]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize, max_features = 1900, max_df = 1.0, ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(use_idf = False)),
    ('clf', MultiOutputClassifier(KNeighborsClassifier( leaf_size = 30 )))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
colnames = Y.columns
print_class_report(y_test.values, y_pred, colnames)

TypeError: __init__() got an unexpected keyword argument 'random_state'

### Create customer Transformer - Use genre as input

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = pos_tag(word_tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0][0], pos_tags[0][1]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(lambda x: self.starting_verb(x)).values
        return pd.DataFrame(X_tagged)
    
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    return pipeline

In [ ]:
pd.get_dummies()

### 9. Export your model as a pickle file

In [ ]:



#outfile = open('classifier_kn.pkl','rb')
#loaded_model = pickle.load(outfile)
#outfile.close()

dump(cv, 'classifier.pkl')

#loaded_model = load('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.